In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from itertools import combinations
import random

In [ ]:
def load_cities(country):
    """Load cities from a CSV file based on the selected country."""
    return pd.read_csv(f'cities/{country.lower()}.csv', header=None, names=['City', 'lat', 'lon'])

def create_distance_matrix(cities):
    """Create a distance matrix for the cities based on their lat/lon coordinates."""
    DIST_MATRIX = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return DIST_MATRIX

countries = ["Vanuatu", "Italy", "Russia", "US", "China"]

In [ ]:
def compute_distance(route, distance_matrix):
    """Calculate the total distance of a given route, ensuring it is a closed loop."""
    return sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))

#2-OPT
2-opt is an algorithm which was invented to solve the TSP problem (Source: https://slowandsteadybrain.medium.com/traveling-salesman-problem-ce78187cf1f3). Because this algorithm provides easily a good solution to TSP, my idea was to combine it to other algorithm.

Why did I decide to use 2-opt? The focus of this lab was genetic algorithms, so that was what I applied at first. But I got results which were worse than a simple greedy algorithm, so I decided to look for something different to combine to the genetic algorithm.

*For all the algorithms there's the output for all the countries. Instead of the name of the city, there's its row number in the CSV file. I put the full route in the output (and in my computations) meaning that the last city is also the first one.*

In [ ]:
def two_opt(route, distance_matrix):
    """Perform 2-opt to improve the route by swapping two edges to reduce distance."""
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                if distance_matrix[route[i - 1]][route[i]] + distance_matrix[route[j]][route[j + 1]] > \
                   distance_matrix[route[i - 1]][route[j]] + distance_matrix[route[i]][route[j + 1]]:
                    route[i:j + 1] = route[i:j + 1][::-1]  # Perform the swap
                    improved = True
    return route

#Greedy + 2-OPT
As seen in class, the Greedy Algorithm provides a fast (and "dirty") solution. I wanted to have this in my code to be sure that the more complex algorithms were actually giving good results! I also gave as an output the greedy algorithm alone, and we can see that adding 2-opt makes the results so much better (and the algorithm is still pretty fast)

Tot computation time: 90 seconds

In [ ]:
def greedy(distance_matrix):
    n = len(distance_matrix)
    visited = [False] * n
    tour = [0]
    visited[0] = True
    total_distance = 0

    for _ in range(1, n):
        last_city = tour[-1]
        nearest_city = None
        nearest_distance = float('inf')

        for city in range(n):
            if not visited[city] and distance_matrix[last_city][city] < nearest_distance:
                nearest_distance = distance_matrix[last_city][city]
                nearest_city = city

        tour.append(nearest_city)
        visited[nearest_city] = True
        total_distance += nearest_distance

    # Return to the starting city
    total_distance += distance_matrix[tour[-1]][tour[0]]
    tour.append(tour[0])  # Completing the tour by returning to the start

    # Initial Greedy solution
    initial_tour = tour[:]
    initial_distance = total_distance

    # Apply 2-opt to improve the solution
    optimized_tour = two_opt(tour, distance_matrix)
    optimized_distance = compute_distance(optimized_tour, distance_matrix)

    return initial_tour, initial_distance, optimized_tour, optimized_distance

In [ ]:
def output_greedy(country):
    cities = load_cities(country)
    distance_matrix = create_distance_matrix(cities)

    initial_tour, initial_distance, optimized_tour, optimized_distance = greedy(distance_matrix)

    print(f"Initial Greedy TSP Tour for {country}: {initial_tour}")
    print(f"Initial Total Distance: {initial_distance:.2f} km")
    print("\n")
    print(f"Optimized TSP Tour with 2-Opt for {country}: {optimized_tour}")
    print(f"Optimized Total Distance: {optimized_distance:.2f} km")

for country in countries:
    print(f"Processing data for {country}...")
    output_greedy(country)
    print("\n")

Processing data for Vanuatu...
Initial Greedy TSP Tour for Vanuatu: [0, 7, 1, 4, 3, 5, 2, 6, 0]
Initial Total Distance: 1475.53 km


Optimized TSP Tour with 2-Opt for Vanuatu: [0, 7, 1, 4, 3, 5, 6, 2, 0]
Optimized Total Distance: 1345.54 km


Processing data for Italy...
Initial Greedy TSP Tour for Italy: [0, 33, 12, 30, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 44, 45, 23, 43, 41, 5, 40, 22, 42, 13, 16, 29, 10, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 38, 17, 31, 8, 37, 24, 7, 36, 27, 0]
Initial Total Distance: 4436.03 km


Optimized TSP Tour with 2-Opt for Italy: [0, 41, 43, 23, 45, 5, 40, 44, 6, 28, 3, 20, 18, 22, 42, 13, 25, 32, 19, 4, 9, 30, 33, 12, 10, 29, 16, 36, 7, 24, 8, 37, 31, 17, 38, 2, 1, 11, 35, 21, 14, 15, 34, 39, 26, 27, 0]
Optimized Total Distance: 4263.21 km


Processing data for Russia...
Initial Greedy TSP Tour for Russia: [0, 54, 1, 40, 58, 109, 86, 15, 11, 90, 139, 124, 113, 97, 137, 142, 55, 49, 20, 65, 166, 105, 159, 36, 79, 104, 14, 33, 74, 70, 77, 2, 96, 143, 131, 117,

#SA + 2-OPT
Simulated annealing combined with 2-opt

Tot computation time: 1 hour and 15 minutes

In [ ]:
def simulated_annealing(distance_matrix, initial_temp=10000, cooling_rate=0.995, min_temp=1):

    n = len(distance_matrix)
    # Initial solution: random route
    current_route = list(range(n))
    random.shuffle(current_route)
    current_route.append(current_route[0])  # Complete the loop

    best_route = current_route[:]
    best_distance = compute_distance(current_route, distance_matrix)
    current_temp = initial_temp

    while current_temp > min_temp:
        # Generate a neighbor by swapping two cities
        new_route = current_route[:]
        i, j = sorted(random.sample(range(1, n), 2))
        new_route[i:j+1] = reversed(new_route[i:j+1])

        # Apply 2-opt to the new route
        new_route = two_opt(new_route, distance_matrix)
        new_distance = compute_distance(new_route, distance_matrix)

        # Acceptance probability
        if new_distance < best_distance or random.random() < np.exp((best_distance - new_distance) / current_temp):
            current_route = new_route
            current_distance = new_distance
            if current_distance < best_distance:
                best_route, best_distance = current_route, current_distance

        current_temp *= cooling_rate

    return best_route, best_distance

In [ ]:
def output_sa(country):
    cities = load_cities(country)
    distance_matrix = create_distance_matrix(cities)
    tour, total_distance = simulated_annealing(distance_matrix)

    print(f"Simulated Annealing + 2-opt TSP Tour for {country}: {tour}")
    print(f"Total Distance: {total_distance:.2f} km")

for country in countries:
    print(f"Processing data for {country} with Simulated Annealing + 2-opt...")
    output_sa(country)
    print("\n")

Processing data for Vanuatu with Simulated Annealing + 2-opt...
Simulated Annealing + 2-opt TSP Tour for Vanuatu: [7, 0, 2, 6, 5, 3, 4, 1, 7]
Total Distance: 1345.54 km


Processing data for Italy with Simulated Annealing + 2-opt...
Simulated Annealing + 2-opt TSP Tour for Italy: [18, 22, 42, 13, 28, 25, 32, 19, 4, 29, 10, 16, 36, 7, 24, 8, 37, 31, 17, 38, 2, 1, 11, 35, 21, 14, 15, 34, 39, 26, 27, 0, 33, 12, 30, 9, 23, 43, 41, 5, 40, 45, 44, 6, 3, 20, 18]
Total Distance: 4172.76 km


Processing data for Russia with Simulated Annealing + 2-opt...
Simulated Annealing + 2-opt TSP Tour for Russia: [15, 86, 109, 58, 40, 139, 124, 1, 0, 57, 54, 17, 3, 31, 144, 23, 16, 151, 71, 6, 146, 41, 48, 162, 106, 61, 157, 82, 92, 93, 78, 132, 75, 137, 142, 36, 159, 105, 79, 104, 14, 133, 4, 123, 67, 107, 116, 47, 148, 110, 34, 147, 127, 18, 102, 56, 13, 89, 125, 84, 112, 12, 135, 53, 88, 128, 64, 5, 130, 76, 22, 45, 111, 72, 149, 73, 30, 42, 63, 38, 24, 8, 28, 152, 153, 155, 37, 29, 120, 9, 134, 85, 11

#GA + 2-OPT
Genetic algorithm combined with 2-opt. The problem with GA is that the population and generation numbers should be tailored for every specific problem, which is something I didn't do because the China TSP takes already a long time to run with the low numbers I chose. I think that GA + 2-opt + a better selection of the parameter is the best approach to actually find the optimal solutions for every scenario.

Tot computation time: 30 minutes

In [ ]:
def initialize_population(num_individuals, num_cities):
    """Initialize a population of random TSP routes."""
    population = []
    for _ in range(num_individuals):
        route = list(range(num_cities))
        random.shuffle(route)
        population.append(route)
    return population

def tournament_selection(population, fitness, k=3):
    """Select a route using tournament selection."""
    selected = random.sample(range(len(population)), k)
    selected = sorted(selected, key=lambda x: fitness[x])
    return population[selected[0]]

def order_crossover(parent1, parent2):
    """Apply order crossover (OX) to generate an offspring from two parents."""
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]

    pos = end
    for city in parent2:
        if city not in child:
            if pos >= len(parent1):
                pos = 0
            child[pos] = city
            pos += 1
    return child

In [ ]:
def genetic_algorithm(distance_matrix, num_generations=100, population_size=50, mutation_rate=0.2):
    num_cities = len(distance_matrix)

    population = [initialize_route(num_cities) for _ in range(population_size)]

    best_route = None
    best_distance = float('inf')

    for generation in range(num_generations):
        # Calculate fitness for each route (loop closure included in distance)
        fitness = [compute_distance(route, distance_matrix) for route in population]

        # Update best route
        min_distance = min(fitness)
        if min_distance < best_distance:
            best_distance = min_distance
            best_route = population[fitness.index(min_distance)]

        # Generate new population
        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)
            child = order_crossover(parent1[:-1], parent2[:-1])  # Perform crossover without the last city

            # Apply mutation (2-opt) with a probability
            if random.random() < mutation_rate:
                child = two_opt(child, distance_matrix)
            child.append(child[0])  # Close the loop

            new_population.append(child)

        population = new_population

    return best_route, best_distance

def initialize_route(num_cities):
    # Initialize a random route
    route = list(range(num_cities))
    random.shuffle(route)
    route.append(route[0])  # Close the loop
    return route


In [ ]:
def output_ga(country):
    cities = load_cities(country)
    distance_matrix = create_distance_matrix(cities)
    tour, total_distance = genetic_algorithm(distance_matrix)

    print(f"GA + 2-opt TSP Tour for {country}: {tour}")
    print(f"Total Distance: {total_distance:.2f} km")

for country in countries:
    print(f"Processing data for {country}...")
    output_ga(country)
    print("\n")

Processing data for Vanuatu...
GA + 2-opt TSP Tour for Vanuatu: [7, 0, 2, 6, 5, 3, 4, 1, 7]
Total Distance: 1345.54 km


Processing data for Italy...
GA + 2-opt TSP Tour for Italy: [19, 4, 29, 10, 16, 36, 7, 24, 8, 37, 31, 17, 38, 2, 1, 11, 35, 21, 14, 15, 34, 39, 26, 27, 0, 33, 12, 30, 9, 23, 43, 41, 5, 40, 45, 44, 6, 3, 20, 18, 22, 42, 13, 28, 25, 32, 19]
Total Distance: 4172.76 km


Processing data for Russia...
GA + 2-opt TSP Tour for Russia: [120, 9, 134, 85, 118, 138, 25, 145, 39, 83, 19, 161, 44, 33, 70, 77, 2, 96, 74, 143, 131, 117, 99, 100, 91, 101, 62, 166, 65, 20, 49, 55, 97, 113, 15, 11, 90, 124, 139, 40, 58, 109, 86, 1, 54, 0, 57, 17, 3, 31, 144, 23, 16, 146, 6, 151, 71, 41, 48, 162, 106, 61, 157, 82, 92, 93, 78, 132, 75, 137, 142, 36, 159, 105, 79, 104, 14, 133, 4, 123, 67, 107, 116, 47, 148, 110, 34, 147, 127, 18, 102, 87, 140, 35, 94, 122, 108, 95, 163, 43, 66, 60, 69, 50, 126, 10, 164, 165, 46, 114, 98, 27, 81, 121, 141, 21, 154, 115, 158, 51, 32, 52, 150, 68, 26, 80, 